In [1]:
import numpy as np
import pandas as pd

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora, models

# Настраиваем NLTK

In [2]:
nltk.download('punkt')
nltk.download('stopwords')
stop_words = stopwords.words("russian");

[nltk_data] Downloading package punkt to /home/kostanew/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/kostanew/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Делаем для одной строки

In [3]:
full_data = pd.read_csv('full_data.csv').full
test_string = full_data[0]
test_string

'Повышение качества и производительности в процессе индустриального жилищного строительства путем внедрения современных технологий на примере ООО ХХХХ ООО XXXX является дочерним предприятием группы компаний ПАО XXXX.\nОсновными сферами деятельности является новое жилищное многоэтажное строительство, реконструкция и новое строительство производственных зданий и сооружений, а также благоустройство и озеленение общественных пространств. На сегодняшний день на предприятии трудятся 50 человек основного штата. Объемы выручки неуклонно растут каждый год, так по данным ресурса «Rusprofile.ru» объемы выручки: 2017 год – 95 млн.руб., 2018 год – 115 млн.руб.,\n2019 год – 177 млн.руб. Ожидаемая выручка за 2020 год более 220 млн.руб. Автор данной работы является руководителем директором ООО XXXX. Проблемой предприятия, на решение которой и направлена данная дипломная работа, является недостаточная прибыльность компании, вызванная низкой компетентностью штатного персонала, срывами сроков производств

In [4]:
tokens = word_tokenize(test_string, language="russian")
tokens

['Повышение',
 'качества',
 'и',
 'производительности',
 'в',
 'процессе',
 'индустриального',
 'жилищного',
 'строительства',
 'путем',
 'внедрения',
 'современных',
 'технологий',
 'на',
 'примере',
 'ООО',
 'ХХХХ',
 'ООО',
 'XXXX',
 'является',
 'дочерним',
 'предприятием',
 'группы',
 'компаний',
 'ПАО',
 'XXXX',
 '.',
 'Основными',
 'сферами',
 'деятельности',
 'является',
 'новое',
 'жилищное',
 'многоэтажное',
 'строительство',
 ',',
 'реконструкция',
 'и',
 'новое',
 'строительство',
 'производственных',
 'зданий',
 'и',
 'сооружений',
 ',',
 'а',
 'также',
 'благоустройство',
 'и',
 'озеленение',
 'общественных',
 'пространств',
 '.',
 'На',
 'сегодняшний',
 'день',
 'на',
 'предприятии',
 'трудятся',
 '50',
 'человек',
 'основного',
 'штата',
 '.',
 'Объемы',
 'выручки',
 'неуклонно',
 'растут',
 'каждый',
 'год',
 ',',
 'так',
 'по',
 'данным',
 'ресурса',
 '«',
 'Rusprofile.ru',
 '»',
 'объемы',
 'выручки',
 ':',
 '2017',
 'год',
 '–',
 '95',
 'млн.руб.',
 ',',
 '2018',
 'г

In [5]:
# Удаляем все знаки препинания
tokens = [word for word in tokens if word.isalnum()]
# Удаляем все стоп-слова (предлоги и т.д.) 
tokens = [token for token in tokens if token not in stop_words]
# Удаляем окончания
snowball = SnowballStemmer(language="russian")
tokens = [snowball.stem(token) for token in tokens]

# Собираем обратно в строку, чтобы потом выделять строки
string_for_analysis = ' '.join(tokens)
string_for_analysis

'повышен качеств производительн процесс индустриальн жилищн строительств пут внедрен современ технолог пример оо хххх оо XXXX явля дочерн предприят групп компан па XXXX основн сфер деятельн явля нов жилищн многоэтажн строительств реконструкц нов строительств производствен здан сооружен такж благоустройств озеленен обществен пространств на сегодняшн ден предприят труд 50 человек основн штат объем выручк неуклон растут кажд год дан ресурс объем выручк 2017 год 95 2018 год 115 2019 год 177 ожида выручк 2020 год 220 автор дан работ явля руководител директор оо XXXX проблем предприят решен котор направл дан дипломн работ явля недостаточн прибыльн компан вызва низк компетентн штатн персона срыв срок производств работ низк производительн для реализац план предприят необходим произвест оптимизац корректировк планирован работ модернизац сист производствен цикл решен вышеизложен пробл оо XXXX явля повышен качеств производительн оо XXXX счет оптимизац систем работ поставщик применен современ мето

In [6]:
indexes = []
text_collection = []
tokenized_collection = []

for idx,test_string in enumerate(full_data.values):
    if isinstance(test_string, str):
        tokens = word_tokenize(test_string, language="russian")
        # Удаляем все знаки препинания
        tokens = [word for word in tokens if word.isalnum()]
        # Удаляем все стоп-слова (предлоги и т.д.) 
        tokens = [token for token in tokens if token not in stop_words]
        # Удаляем окончания
        snowball = SnowballStemmer(language="russian")
        tokens = [snowball.stem(token) for token in tokens]
        
        tokenized_collection.append(tokens)

        # Собираем обратно в строку, чтобы потом выделять строки
        string_for_analysis = ' '.join(tokens)
        string_for_analysis

        text_collection.append(string_for_analysis)
        indexes.append(idx)

In [7]:
len(tokenized_collection), len(text_collection), len(indexes)

(270, 270, 270)

# LDA+BoW

In [8]:
dictionary = gensim.corpora.Dictionary(tokenized_collection)
dictionary

In [9]:
bow_corpus = [dictionary.doc2bow(doc) for doc in tokenized_collection]
bow_corpus

[[(0, 1),
  (1, 1),
  (2, 1),
  (3, 1),
  (4, 1),
  (5, 1),
  (6, 1),
  (7, 1),
  (8, 1),
  (9, 1),
  (10, 1),
  (11, 1),
  (12, 1),
  (13, 5),
  (14, 1),
  (15, 3),
  (16, 1),
  (17, 1),
  (18, 1),
  (19, 1),
  (20, 1),
  (21, 1),
  (22, 2),
  (23, 2),
  (24, 1),
  (25, 1),
  (26, 1),
  (27, 3),
  (28, 1),
  (29, 5),
  (30, 1),
  (31, 4),
  (32, 1),
  (33, 1),
  (34, 1),
  (35, 1),
  (36, 1),
  (37, 2),
  (38, 1),
  (39, 1),
  (40, 1),
  (41, 2),
  (42, 1),
  (43, 1),
  (44, 1),
  (45, 1),
  (46, 1),
  (47, 1),
  (48, 1),
  (49, 1),
  (50, 1),
  (51, 2),
  (52, 4),
  (53, 1),
  (54, 1),
  (55, 1),
  (56, 1),
  (57, 1),
  (58, 1),
  (59, 2),
  (60, 1),
  (61, 1),
  (62, 1),
  (63, 2),
  (64, 1),
  (65, 2),
  (66, 2),
  (67, 2),
  (68, 1),
  (69, 3),
  (70, 2),
  (71, 1),
  (72, 1),
  (73, 5),
  (74, 1),
  (75, 1),
  (76, 1),
  (77, 2),
  (78, 2),
  (79, 1),
  (80, 1),
  (81, 1),
  (82, 3),
  (83, 1),
  (84, 2),
  (85, 1),
  (86, 1),
  (87, 2),
  (88, 1),
  (89, 5),
  (90, 1),
  (91, 1)

In [10]:
bow_doc_100 = bow_corpus[100] 

for i in range(len(bow_doc_100)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_100[i][0], 
                                               dictionary[bow_doc_100[i][0]], 
bow_doc_100[i][1]))

Word 7 ("5") appears 2 time.
Word 11 ("PESTEL") appears 1 time.
Word 13 ("XXXX") appears 2 time.
Word 14 ("автор") appears 1 time.
Word 15 ("анализ") appears 4 time.
Word 21 ("в") appears 3 time.
Word 22 ("включа") appears 1 time.
Word 23 ("внедрен") appears 3 time.
Word 24 ("внутрен") appears 1 time.
Word 30 ("групп") appears 1 time.
Word 31 ("дан") appears 1 time.
Word 34 ("деятельн") appears 2 time.
Word 35 ("дипломн") appears 1 time.
Word 37 ("для") appears 1 time.
Word 43 ("здан") appears 2 time.
Word 44 ("издержек") appears 7 time.
Word 46 ("индустриальн") appears 1 time.
Word 47 ("инструмент") appears 1 time.
Word 49 ("кажд") appears 2 time.
Word 51 ("качеств") appears 1 time.
Word 52 ("компан") appears 24 time.
Word 55 ("котор") appears 1 time.
Word 60 ("направл") appears 1 time.
Word 63 ("необходим") appears 1 time.
Word 65 ("низк") appears 1 time.
Word 67 ("обеспечен") appears 1 time.
Word 68 ("обществен") appears 5 time.
Word 78 ("основн") appears 1 time.
Word 83 ("планирова

In [11]:
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]

In [16]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=7, id2word=dictionary, passes=2, workers=4)

for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}\n'.format(idx, topic))

Topic: 0 Word: 0.001*"оо" + 0.001*"оа" + 0.001*"компан" + 0.001*"а" + 0.001*"продукц" + 0.001*"европейск" + 0.001*"сфер" + 0.001*"издел" + 0.001*"территор" + 0.001*"обслуживан"

Topic: 1 Word: 0.001*"учрежден" + 0.001*"оа" + 0.001*"услуг" + 0.001*"образовательн" + 0.001*"профессиональн" + 0.001*"компан" + 0.001*"свердловск" + 0.001*"фгуп" + 0.001*"юридическ" + 0.001*"комбинат"

Topic: 2 Word: 0.001*"оборудован" + 0.001*"труб" + 0.001*"компан" + 0.001*"услуг" + 0.001*"продукц" + 0.001*"со" + 0.001*"строительн" + 0.001*"продаж" + 0.001*"отдел" + 0.001*"медицинск"

Topic: 3 Word: 0.001*"труб" + 0.001*"оо" + 0.001*"оборудован" + 0.001*"услуг" + 0.001*"компан" + 0.001*"производств" + 0.001*"издержек" + 0.001*"оа" + 0.001*"план" + 0.001*"недвижим"

Topic: 4 Word: 0.002*"труб" + 0.001*"производств" + 0.001*"сыр" + 0.001*"оа" + 0.001*"продукц" + 0.001*"продаж" + 0.001*"завод" + 0.001*"трубн" + 0.001*"систем" + 0.001*"а"

Topic: 5 Word: 0.001*"школ" + 0.001*"картофел" + 0.001*"образовательн" + 

In [17]:
bow_corpus

[[(0, 1),
  (1, 1),
  (2, 1),
  (3, 1),
  (4, 1),
  (5, 1),
  (6, 1),
  (7, 1),
  (8, 1),
  (9, 1),
  (10, 1),
  (11, 1),
  (12, 1),
  (13, 5),
  (14, 1),
  (15, 3),
  (16, 1),
  (17, 1),
  (18, 1),
  (19, 1),
  (20, 1),
  (21, 1),
  (22, 2),
  (23, 2),
  (24, 1),
  (25, 1),
  (26, 1),
  (27, 3),
  (28, 1),
  (29, 5),
  (30, 1),
  (31, 4),
  (32, 1),
  (33, 1),
  (34, 1),
  (35, 1),
  (36, 1),
  (37, 2),
  (38, 1),
  (39, 1),
  (40, 1),
  (41, 2),
  (42, 1),
  (43, 1),
  (44, 1),
  (45, 1),
  (46, 1),
  (47, 1),
  (48, 1),
  (49, 1),
  (50, 1),
  (51, 2),
  (52, 4),
  (53, 1),
  (54, 1),
  (55, 1),
  (56, 1),
  (57, 1),
  (58, 1),
  (59, 2),
  (60, 1),
  (61, 1),
  (62, 1),
  (63, 2),
  (64, 1),
  (65, 2),
  (66, 2),
  (67, 2),
  (68, 1),
  (69, 3),
  (70, 2),
  (71, 1),
  (72, 1),
  (73, 5),
  (74, 1),
  (75, 1),
  (76, 1),
  (77, 2),
  (78, 2),
  (79, 1),
  (80, 1),
  (81, 1),
  (82, 3),
  (83, 1),
  (84, 2),
  (85, 1),
  (86, 1),
  (87, 2),
  (88, 1),
  (89, 5),
  (90, 1),
  (91, 1)